In [1]:
import cffi
import numpy as np
import pandas as pd

In [2]:
with open("TRS-omix.c", "r") as f:
    lines = f.read()



ffi = cffi.FFI()
ffi.set_source("_trs_omix_python", lines)
ffi.cdef(""" 
void PrintError(int e);
typedef struct {...;} NPt;
typedef struct {...;} NLt;
long long int ImportGenome(char *fn, NLt *nlt);
char * PerformTRSCalculation(char *gfn, char* tfn, char* ifn, long tmin, long tmax, int mode);
""")

ffi.compile(verbose=0)

_trs_omix_python.c: In function ‘PrtNLtToBuffer’:
_trs_omix_python.c:912:11: warning: assignment makes integer from pointer without a cast [-Wint-conversion]
         l = strcat(buffer, line);
           ^
_trs_omix_python.c: In function ‘FindFistrTRSPosition’:
_trs_omix_python.c:1234:10: warning: unused variable ‘npt3’ [-Wunused-variable]
     NPt *npt3 = nlt->head, *npt1 = nlt->head;
          ^~~~
_trs_omix_python.c: In function ‘LC_InteriorsFindAndSaveToFile’:
_trs_omix_python.c:1684:24: warning: format not a string literal and no format arguments [-Wformat-security]
         if (fprintf(f, header) < 0) {
                        ^~~~~~
_trs_omix_python.c:1745:49: warning: format ‘%d’ expects argument of type ‘int’, but argument 4 has type ‘long long int’ [-Wformat=]
                         l = sprintf(line, ";%d;%d;%d;0%d;0%d;",abs(ctvpt->pe - ctvpt->ps + 1),ctvpt->ps,ctvpt->pe,ctvpt->next->cl,ctvpt->next->no);
                                                ~^                    

'/home/rafalb/molecules/TRS-omix/TRS-omix/python/_trs_omix_python.cpython-39-x86_64-linux-gnu.so'

In [3]:
from _trs_omix_python import ffi, lib


In [26]:
gfn = b"sequence.fasta"
pGfn = ffi.cast("char *", ffi.from_buffer(gfn))

tfn = b"trs.txt"
pTfn = ffi.cast("char *", ffi.from_buffer(tfn))

ifn = b"interiors.csv"
pIfn = ffi.cast("char *", ffi.from_buffer(ifn))

# tmin = float(3.0)
tmin = 3
tmin = ffi.cast("long long", tmin)

# tmax = float(6.0)
tmax = 20
tmax = ffi.cast("long long", tmax)

mode = 0
mode = ffi.cast("int", mode)

f_num = 10.0
f_num = ffi.cast("float *", f_num)

result = lib.PerformTRSCalculation(pGfn, pTfn, pIfn, tmin, tmax, mode)


name of genome file: sequence.fasta
name of input file: trs.txt
name of output file: interiors.csv
tmin: 3
tmax: 20
mode: 0

START

size of genome: 5498578
size of input: 9
status after LC_TRSPositionsFindAndSaveToVLt: 1file_content outside: L-NoClass;L-No;LFS;Len(LFS);L-POS(LFS);R-POS(LFS);R-NoClass;R-No;RFS;Len(RFS);L-POS(RFS);R-POS(RFS);>SEQ;Len(SEQ)
02;05;GGCGGCGGC;9;14422;14430;02;05;GGCGGCGGC;9;14437;14445;>GGTTTT;6
02;06;GCGGCGGCG;9;179850;179858;02;05;GGCGGCGGC;9;179864;179872;>GGTCT;5
10;30;TGCTGCTGC;9;244586;244594;03;08;CCACCACCA;9;244609;244617;>ACCGGATAGACCTG;14
03;08;CCACCACCA;9;253225;253233;02;04;CGGCGGCGG;9;253248;253256;>CCGGGTTGTTTTTA;14
01;01;CCGCCGCCG;9;463672;463680;04;12;TGGTGGTGG;9;463684;463692;>GGC;3
17;51;CATCATCATCAT;12;483444;483455;17;51;CATCATCAT;9;483474;483482;>GATTGTCGCCGGTTTCGG;18
19;57;AATAATAAT;9;799415;799423;19;56;TAATAATAA;9;799438;799446;>ATTTGGCGCAGGTT;14
08;22;CGTCGTCGT;9;832074;832082;18;53;ATGATGATG;9;832096;832104;>AGGTATTCGCCAC;13
10;30;T

In [9]:
ffi.list_types()

(['NLt', 'NPt'], [], [])

In [27]:
result = ffi.string(result).decode("utf-8")

In [21]:
type(result)

bytes

In [24]:
from io import StringIO

In [29]:
StringIO(result)

In [31]:
pd.read_csv(StringIO(result), sep=";")

,L-NoClass,L-No,LFS,Len(LFS),L-POS(LFS),R-POS(LFS),R-NoClass,R-No,RFS,Len(RFS),L-POS(RFS),R-POS(RFS),>SEQ,Len(SEQ)
0,2,5,GGCGGCGGC,9,14422,14430,2,5,GGCGGCGGC,9,14437,14445,>GGTTTT,6
1,2,6,GCGGCGGCG,9,179850,179858,2,5,GGCGGCGGC,9,179864,179872,>GGTCT,5
2,10,30,TGCTGCTGC,9,244586,244594,3,8,CCACCACCA,9,244609,244617,>ACCGGATAGACCTG,14
3,3,8,CCACCACCA,9,253225,253233,2,4,CGGCGGCGG,9,253248,253256,>CCGGGTTGTTTTTA,14
4,1,1,CCGCCGCCG,9,463672,463680,4,12,TGGTGGTGG,9,463684,463692,>GGC,3
5,17,51,CATCATCATCAT,12,483444,483455,17,51,CATCATCAT,9,483474,483482,>GATTGTCGCCGGTTTCGG,18
6,19,57,AATAATAAT,9,799415,799423,19,56,TAATAATAA,9,799438,799446,>ATTTGGCGCAGGTT,14
7,8,22,CGTCGTCGT,9,832074,832082,18,53,ATGATGATG,9,832096,832104,>AGGTATTCGCCAC,13
8,10,30,TGCTGCTGC,9,902048,902056,4,12,TGGTGGTGG,9,902060,902068,>TAT,3
9,2,5,GGCGGCGGC,9,954422,954430,4,10,GGTGGTGGTGGT,12,954443,954454,>GGTCGTGGGCAA,12


In [5]:
pd.read_csv(ifn.decode("utf-8"), sep=";")

,L-NoClass,L-No,LFS,Len(LFS),L-POS(LFS),R-POS(LFS),R-NoClass,R-No,RFS,Len(RFS),L-POS(RFS),R-POS(RFS),>SEQ,Len(SEQ)
0,2,5,GGCGGCGGC,9,14422,14430,2,5,GGCGGCGGC,9,14437,14445,>GGTTTT,6
1,2,6,GCGGCGGCG,9,179850,179858,2,5,GGCGGCGGC,9,179864,179872,>GGTCT,5
2,10,30,TGCTGCTGC,9,244586,244594,3,8,CCACCACCA,9,244609,244617,>ACCGGATAGACCTG,14
3,3,8,CCACCACCA,9,253225,253233,2,4,CGGCGGCGG,9,253248,253256,>CCGGGTTGTTTTTA,14
4,1,1,CCGCCGCCG,9,463672,463680,4,12,TGGTGGTGG,9,463684,463692,>GGC,3
5,17,51,CATCATCATCAT,12,483444,483455,17,51,CATCATCAT,9,483474,483482,>GATTGTCGCCGGTTTCGG,18
6,19,57,AATAATAAT,9,799415,799423,19,56,TAATAATAA,9,799438,799446,>ATTTGGCGCAGGTT,14
7,8,22,CGTCGTCGT,9,832074,832082,18,53,ATGATGATG,9,832096,832104,>AGGTATTCGCCAC,13
8,10,30,TGCTGCTGC,9,902048,902056,4,12,TGGTGGTGG,9,902060,902068,>TAT,3
9,2,5,GGCGGCGGC,9,954422,954430,4,10,GGTGGTGGTGGT,12,954443,954454,>GGTCGTGGGCAA,12


In [17]:
from Bio import SeqIO
from Bio.Seq import Seq

In [18]:
for seq_record in SeqIO.parse(gfn.decode("utf-8"), "fasta"):
    print(seq_record.id)
    print(repr(seq_record.seq))
    print(len(seq_record))
    print(seq_record.annotations)
    print(seq_record.features)

BA000007.3
Seq('AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAG...TTC')
5498578
{}
[]


In [26]:
seq = SeqIO.parse(gfn.decode("utf-8"), "fasta")

In [27]:
list(seq)

[SeqRecord(seq=Seq('AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAG...TTC'), id='BA000007.3', name='BA000007.3', description='BA000007.3 Escherichia coli O157:H7 str. Sakai DNA, complete genome', dbxrefs=[])]